# DOPEによる違反なしの探索

**コメント**：多分Optimismのところ間違ってるかも．DOPEでは推定した$\widehat{P}$を中心とした信頼区間からOptimisticな$P$も線型計画法で計算するが，そのときにValidな$P$が出てくる保証がなさそう．確率分布が出てこないと，Occupancy measureの存在も保証されなさそうである．

参考
* [DOPE: Doubly Optimistic and Pessimistic Exploration for Safe Reinforcement Learning](https://arxiv.org/abs/2112.00885)


表記
* 論文参照．また，[RL_CMDP_explore_exploit.ipynb](RL_CMDP_explore_exploit.ipynb)とだいたい同じ
* $\pi^0$は$V_1^{\pi^0}(\mu ; c, P)=c^0<\tau$を満たすときにStrictly safeといいます．
* $\Pi_{\text {safe }}=\left\{\pi: V_c^\pi(P) \leq \bar{C}\right\}$とします．

次のCMDPを考えます．報酬とコストは既知とします．

$$
\min _\pi \quad V_r^\pi(P) \quad \text { s.t. } \quad V_c^\pi(P) \leq \bar{C}
$$

仮定
* $V_c^{\pi_b}(P)=\bar{C}_b$, where $\bar{C}_b<\bar{C}$を満たす$\pi_b$が存在するとします．また，$C_b$は既知とします．
* 多分$\bar{C}_b$は既知じゃなくても，最初に$\pi_b$を数回走らせて計算すれば良いです．


## Good Event

**遷移についての信頼区間**

$$
\begin{aligned}
& \beta_{h, k}^p\left(s, a, s^{\prime}\right)=\sqrt{\frac{4 \operatorname{Var}\left(\widehat{P}_{h, k}\left(s^{\prime} \mid s, a\right)\right) \log \left(\frac{2 S A K H}{\delta}\right)}{n_{h, k}(s, a) \vee 1}}+\frac{14 \log \left(\frac{2 S A K H}{\delta}\right)}{\left.3\left(n_{h, k}(s, a) \vee 1\right)\right)} \\
& \mathcal{P}_{h, k}(s, a)=\left\{P^{\prime}:\left|P_h^{\prime}\left(s^{\prime} \mid s, a\right)-\widehat{P}_{h, k}\left(s^{\prime} \mid s, a\right)\right| \leq \beta_{h, k}^p\left(s, a, s^{\prime}\right), \forall s^{\prime} \in \mathcal{S}\right\}
\end{aligned}
$$

を遷移についての信頼区間とします．Eventを
$$
F^p=\left\{P \in \mathcal{P}_k, \forall k \in[K]\right\}
$$
とすると，Bernsteinの不等式から次が言えます．
$$
\mathbb{P}\left(F^p\right) \geq 1-2 \delta
$$

**Nice Episode**

$$
\begin{aligned}
& F^w \\
& =\left\{n_{h, k}(s, a) \geq \frac{1}{2} \sum_{j<k} w_{h, j}(s, a)-H \log \frac{S A H}{\delta}, \forall\left(h, s, a, s^{\prime}, k\right) \in[H] \times \mathcal{S} \times \mathcal{A} \times \mathcal{S} \times[K]\right\}
\end{aligned}
$$

とします．ここで$w_{h, j}$はエピソード$j$でのOccupancy Measureです．
このとき，[RL_useful_lemma.ipynb](RL_useful_lemma.ipynb)でやったように，$\mathbb{P}\left(F^w\right) \geq 1-\delta$が成り立ちます．

以上を踏まえて，GoodEventを$G=F^p \cap F^w$とすると，$\mathbb{P}(G) \geq 1-3 \delta$が成り立ちます．


## アルゴリズム

**報酬とコスト**

DOPEでは次の報酬とコストを考えます．

* $\bar{\beta}_{h, k}^p(s, a)=\sum_{s^{\prime} \in \mathcal{S}} \beta_{h, k}^p\left(s, a, s^{\prime}\right)$とします
* コスト：$\bar{c}_{h, k}(s, a)={c}_{h}(s, a) + H \bar{\beta}_{h, k}^p(s, a)$ 
* 報酬：$\bar{r}_{h, k}(s, a)={r}_{h}(s, a)-\frac{H^2}{\bar{C}-\bar{C}_b} \bar{\beta}_{h, k}^p(s, a)$.

そして，方策を

$$
\left(\pi_k, P_k\right)=\underset{\pi^{\prime}, P^{\prime} \in \mathcal{P}_k}{\arg \min } V_{\bar{r}_k}^{\pi^{\prime}}\left(P^{\prime}\right) \text { s.t. } V_{\bar{c}_k}^{\pi^{\prime}}\left(P^{\prime}\right) \leq \bar{C} .
$$

によって選択します．

**違反なしの工夫**

最初の$K_o$エピソードは安全な方策$\pi_b$によって十分なデータを集めます．ここで$K_o=\tilde{\mathcal{O}}\left(\frac{S^2 A H^4}{\left(\bar{C}-\bar{C}_b\right)^2}\right)$とします．

## 便利な補題

準備
* $\epsilon_k^{\pi}\left(P\right)=H \mathbb{E}\left[\sum_{h=1}^H \bar{\beta}_{h, k}^p\left(s_{h, k}, a_{h, k}\right) \mid \pi, P, \mathcal{F}_{k-1}\right]$
* $V_{\bar{c}_k}^{\pi}\left(P\right)=V_{{c}}^{\pi}\left(P\right)+\epsilon_k^{\pi}(P)$
* $V_{\bar{r}_k}^{\pi}\left(P\right)=V_{{r}}^{\pi}\left(P\right)-\frac{H}{\bar{C}-\bar{C}_b} \epsilon_k^{\pi}\left(P\right)$

とします．

Good Eventが成立するとき，次が成立します．（Nice Episodeについての話は[RL_useful_lemma.ipynb](RL_useful_lemma.ipynb)に任せます．

---

**補題(Lemma 15)**

$G$が成立するとき，任意の$P' \in \mathcal{P}_k$について，

$$
\left|P_h^{\prime}\left(s^{\prime} \mid s, a\right)-P_h\left(s^{\prime} \mid s, a\right)\right| \leq C_1 \sqrt{\frac{P_h\left(s^{\prime} \mid s, a\right) L}{n_{h, k}(s, a) \vee 1}}+C_2 \frac{L}{n_{h, k}(s, a) \vee 1}
$$

を満たす定数$C_1, C_2 > 0$が存在します．

---

**補題：$\sum_k \epsilon_k$のバウンド**

GoodEventが成立しているとします．
Niceエピソードの話と$\beta^p$の定義から，
$$\sum_{k=1}^{K^{\prime}} \epsilon_k^{\pi_k}(P) \leq \tilde{\mathcal{O}}\left(S \sqrt{A H^4 K^{\prime}}\right)$$
が成立します．

---



**補題：$\pi_b$が実行可能解になるエピソード**

Good Eventが成立するとき，$k \geq K_o$について，$(\pi_b, P)$は

$$
\left(\pi_k, P_k\right)=\underset{\pi^{\prime}, P^{\prime} \in \mathcal{P}_k}{\arg \min } V_{\bar{r}_k}^{\pi^{\prime}}\left(P^{\prime}\right) \text { s.t. } V_{\bar{c}_k}^{\pi^{\prime}}\left(P^{\prime}\right) \leq \bar{C} .
$$

の実行可能解になります．

**証明**

$\pi_b$が実行可能解のとき，
$$
V_{\bar{c}_k}^{\pi_b}\left(P\right)
=V_{{c}}^{\pi_b}\left(P\right)+\epsilon_k^{\pi_b}(P)
=\bar{C}_b+\epsilon_k^{\pi_b}(P) \leq \bar{C}
$$
なので，
$\pi_b$が実行可能解であることの十分条件は

$$
\epsilon_k^{\pi_b}(P) \leq (\bar{C} - \bar{C}_b)
$$

です．
ここで，最初の$k \leq K_o$エピソードでは$\pi_k=\pi_b$であることを思い出しましょう．
よって，
$$\sum_{k=1}^{K_o} \epsilon_k^{\pi_b}(P) \leq \tilde{\mathcal{O}}\left(S \sqrt{A H^4 K_o}\right)$$

が成り立ちます．$k \leq K_o$まで$\epsilon_k^{\pi_b}(P) > (\bar{C} - \bar{C}_b)$であるとしましょう．
すると，

$$K_o(\bar{C} - \bar{C}_b) < \sum_{k=1}^{K_o} \epsilon_k^{\pi_b}(P) \leq \tilde{\mathcal{O}}\left(S \sqrt{A H^4 K_o}\right)$$

この不等式は$K_o \geq \tilde{\mathcal{O}}\left(\frac{S^2 A H^4}{\left(\bar{C}-\bar{C}_b\right)^2}\right)$では成立しません．よって，
任意の$k \geq K_o \geq \tilde{\mathcal{O}}\left(\frac{S^2 A H^4}{\left(\bar{C}-\bar{C}_b\right)^2}\right)$
エピソードについては
$\epsilon_k^{\pi_b}(P) \leq (\bar{C} - \bar{C}_b)$である必要があります．以上から，$(\pi_b, P)$は$k \geq K_o$について実行可能解です．


---

**補題：$\pi_k$は常に安全**

GoodEventが成立しているとします．このとき，任意の$k$について$\pi_k$は安全です．

**証明**

$k \leq K_o$は$\pi_b$を選択するので常に安全です．
$k \geq K_o$を考えてみましょう．

上の補題より，$k \geq K_o$でアルゴリズム中の最適化問題には実行可能解が存在します．
また，$\pi_k$の定義より，$V_{\bar{c}_k}^{\pi_k}\left(P_k\right) \leq \bar{C}$です．

ここで，価値差分方程式より，

$$
\begin{aligned}
V_c^{\pi_k}(P)-V_c^{\pi_k}\left(P_k\right) & =\mathbb{E}\left[\sum_{h=1}^H\left(\left(P_h-P_{h, k}\right)\left(\cdot \mid s_{h, k}, a_{h, k}\right)\right)^{\top} V_{c, h+1}^{\pi_k}(\cdot ; P) \mid \pi_k, P_k, \mathcal{F}_{k-1}\right] \\
& \stackrel{(a)}{\leq} \mathbb{E}\left[\sum_{h=1}^H\left\|\left(\left(P_h-P_{h, k}\right)\left(\cdot \mid s_{h, k}, a_{h, k}\right)\right)\right\|_1\left\|V_{c, h+1}^{\pi_k}(\cdot ; P)\right\|_{\infty} \mid \pi_k, P_k, \mathcal{F}_{k-1}\right] \\
& \stackrel{(b)}{\leq} H \mathbb{E}\left[\sum^H \bar{\beta}_{h, k}^p\left(s_{h, k}, a_{h, k}\right) \mid \pi_k, P_k, \mathcal{F}_{k-1}\right]=\epsilon_k^{\pi_k}\left(P_k\right) .
\end{aligned}
$$

が成り立ちます．(a)はHolderの不等式，(b)はGoodEventを使ってます．
変形して

$$
V_c^{\pi_k}(P) \leq V_c^{\pi_k}\left(P_k\right)+\epsilon_k^{\pi_k}\left(P_k\right)
=V_{\bar{c}_k}^{\pi_k}\left(P_k\right) \leq \bar{C}
$$

が成立します．


---

**補題：Optimism**

$V_{\bar{r}_k}^{\pi_k}\left(P_k\right) \leq V_{\bar{r}_k}^{\pi_k}\left(P\right) \leq V_r^{\pi^*}(P)$が成立します．
$V_{\bar{r}_k}^{\pi_k}\left(P_k\right) \leq V_{\bar{r}_k}^{\pi_k}\left(P\right)$は明らかなので，右側の不等式だけ証明しましょう．

**方針**：$V_{\tilde{r}_k}^{\tilde{\pi}_k}\left(P\right) \leq V_{\tilde{r}_k}^{\tilde{\pi}}(P) \leq V_r^{\pi^*}(P)$を満たすような実行可能解$\widetilde{\pi}$を見つければ良いです．


**証明**

$\tilde{r}_h^k(s, a)=\hat{r}_h(s, a) - b H \bar{\beta}_{h, k}^p(s, a)$として，

$$
\tilde{\pi}_k=\underset{\pi^{\prime}}{\arg \min } V_{\tilde{r}_k}^{\pi^{\prime}}\left(P\right) \text { subject to } V_{\bar{c}_k}^{\pi^{\prime}}\left(P\right) \leq \bar{C},
$$

を考えてみます．これは$b=\frac{H}{\bar{C} - \bar{C}_b}$のときに$\pi_k$と同じです．

ここで，$\alpha_k > 0$について，

$$
\tilde{w}_h(s, a)=(1-\alpha_k) w_h^{\pi_b}(s, a ; P)+\alpha_k w_h^{\pi^*}(s, a ; P)
$$

なるOccupancy measureを考えてみます．Occupancy measureの集合の凸性から，$\tilde{w}=w^{\tilde{\pi}}$である方策$\tilde{\pi}$ が存在します．
ここで，次が成立します：

**ステップ１**

$$
\alpha_k \leq \frac{\bar{C}-\bar{C}_b-\epsilon_k^{\pi_b}(P)}{\bar{C}-\bar{C}_b+\epsilon_k^{\pi^*}(P)-\epsilon_k^{\pi_b}(P)}
$$

ならば$\tilde{\pi}$は
$
V_{\bar{c}_k}^{\tilde{\pi}}\left(P\right) \leq \bar{C}
$を満たします．

**証明**

$$
\begin{aligned}
& V_{\bar{c}_k}^{\tilde{\pi}}(P)=\left(1-\alpha_k\right) V_{\bar{c}_k}^{\pi_b}(P)+\alpha_k V_{\bar{c}_k}^{\pi^*}(P) \\
& =\left(1-\alpha_k\right)\left(V_{{c}}^{\pi_b}(P)+\epsilon_k^{\pi_b}(P)\right)+\alpha_k\left(V_{{c}}^{\pi^*}(P)+\epsilon_k^{\pi^*}(P)\right) \\
& \leq\left(1-\alpha_k\right)\left(\bar{C}_b+\epsilon_k^{\pi_b}(P)\right)+\alpha_k\left(\bar{C}+\epsilon_k^{\pi^*}(P)\right)
\end{aligned}
$$

これを$\leq\bar{C}$として$\alpha_k$について解けば補題が成立します．

**ステップ２**

$b \geq \frac{H}{\bar{C}-\bar{C}_b}$であれば，$V_{\tilde{r}_k}^{\tilde{\pi}_k}\left(P\right) \leq V_r^{\pi^*}(P)$が成立します．

**証明**

ステップ１より，$V_{\tilde{r}_k}^{\tilde{\pi}_k}\left(P\right) \leq V_{\tilde{r}_k}^{\tilde{\pi}}(P)$が成立します．
よって，$V_{\tilde{r}_k}^{\tilde{\pi}}(P) \leq V_r^{\pi^*}(P)$となるような$b$を見つければ良いです．
つまり，

$$
\begin{aligned}
V_{\tilde{r}_k}^{\tilde{\pi}}(P)
&=
V_{r}^{\tilde{\pi}}(P)-b \epsilon_k^{\tilde{\pi}}(P) \\
&=\left(1-\alpha_k\right)\left(V_{{r}}^{\pi_b}(P)-b \epsilon_k^{\pi_b}(P)\right)+\alpha_k\left(V_{{r}}^{\pi^*}(P)-b \epsilon_k^{\pi^*}(P)\right) 
\leq V^{\pi^*}_r(P)
\end{aligned}
$$
を満たす$b$を見つけます．
変形して，

$$
b \geq \frac{V_r^{\pi_b}(P)-V_r^{\pi^*}(P)}{\epsilon_k^{\pi_b}(P)+\frac{\alpha_k}{1-\alpha_k}\epsilon_k^{\pi^*}(P)}
$$

ステップ１の$\alpha_k$を持ってくると，$\frac{\alpha_k}{1-\alpha_k}=\frac{\bar{C}-\bar{C}_b-\epsilon_k^{\pi_b}(P)}{\epsilon_k^{\pi^*}(P)}$
なので，代入して

$$
b \geq 
\frac{V_r^{\pi_b}(P)-V_r^{\pi^*}(P)}{\bar{C}-\bar{C}_b}
$$

であればよいです．
Optimismはこれで明らかに成立します．

**コメント**

$\Pi_k$を実行可能なPessimisticな方策の集合としましょう．$\Pi_k$の中のなにかしらの$\tilde{\pi}$について，

$$
\begin{aligned}
&V_{r}^{\tilde{\pi}}(P)-b \epsilon_k^{\tilde{\pi}}(P) \leq V^{\pi^*}_r(P)\\
\Longrightarrow \quad&\frac{V_{r}^{\tilde{\pi}}(P) - V^{\pi^*}_r(P)}{\epsilon_k^{\tilde{\pi}}(P)} \leq b
\end{aligned}
$$
が成り立てばいいわけですね．ただし，
$\epsilon_k^{\pi}\left(P\right)=H \mathbb{E}\left[\sum_{h=1}^H \bar{\beta}_{h, k}^p\left(s_{h, k}, a_{h, k}\right) \mid \pi, P, \mathcal{F}_{k-1}\right]$
はモデルの信頼半径の累積和を表します．よって，雑に
$$
\begin{aligned}
\frac{V_{r}^{\tilde{\pi}}(P) - V^{\pi^*}_r(P)}{\epsilon_k^{\tilde{\pi}}(P)}
\leq \frac{H}{\epsilon_k^{\tilde{\pi}}(P)} \leq b
\end{aligned}
$$
とすると，たくさんサンプルが集まり，$\epsilon_k^{\tilde{\pi}}(P) \to 0$の場合に$b \to \infty$になってしまいます．
よって，分子の方も，データが集まると$\to 0$になるよう調整したいわけですね．

ステップ１で設計した$\tilde{\pi}$は
$$
\alpha_k \leq \frac{\bar{C}-\bar{C}_b-\epsilon_k^{\pi_b}(P)}{\bar{C}-\bar{C}_b+\epsilon_k^{\pi^*}(P)-\epsilon_k^{\pi_b}(P)}
=\frac{1}{1 +\frac{\epsilon_k^{\pi^*}(P)}{\bar{C}-\bar{C}_b-\epsilon_k^{\pi_b}(P)}}
$$
のうち，最も大きい値を使って$\pi_b$と$\pi^\star$を混合していました．
つまり，サンプルが集まると$\alpha_k\to 1$であり，$\tilde{\pi}$は$\pi^\star$に近づきます．
$\tilde{\pi}$が$\pi^\star$に対して線形？なので，分子と分母の減り方が定数で抑えられているのかも？です．

<!-- ここで，$\pi_b$を十分推定して，$\epsilon_k^{\pi_b}(P) \leq \epsilon$と十分小さくなった時を考えましょう．このとき
$$
\alpha_k \leq \frac{1}{1 +\frac{\epsilon_k^{\pi^*}(P)}{\bar{C}-\bar{C}_b-\epsilon}}
$$ -->

## リグレットのバウンド

$$
R(K)=\sum_{k=1}^{K_o}\left(V_r^{\pi_k}(P)-V_r^{\pi^*}(P)\right)+\sum_{k=K_o}^K\left(V_r^{\pi_k}(P)-V_r^{\pi^*}(P)\right)
$$

であり，$K_o$の定義から，

$$
\sum_{k=1}^{K_o}\left(V_r^{\pi_k}(P)-V_r^{\pi^*}(P)\right) \leq H K_0 \leq \tilde{\mathcal{O}}\left(\frac{S^2 A H^5}{\left(\bar{C}-\bar{C}_b\right)^2}\right)
$$

です．二項目をバウンドしましょう．

$$
\begin{aligned}
\sum_{k=K_o}^K \left(V_r^{\pi_k}(P)-V_r^{\pi^*}(P)\right) & 
\stackrel{(a)}{\leq} \sum_{k=K_o}^K\left(V_r^{\pi_k}(P)-V_{\bar{r}_k}^{\pi_k}\left(P_k\right)\right)\\
& \stackrel{(b)}{\leq} \sum_{k=K_0}^K\left(V_{\bar{r}_k}^{\pi_k}(P)-V_{\bar{r}_k}^{\pi_k}\left(P_k\right)\right)+\frac{H}{\bar{C}-\bar{C}_b} \sum_{k=K_0}^K\epsilon_k^{\pi_k}(P) \\
& \stackrel{(c)}{\leq} \sum_{k=K_o}^K\left(V_{\bar{r}_k}^{\pi_k}(P)-V_{\bar{r}_k}^{\pi_k}\left(P_k\right)\right)+\tilde{O}\left(\frac{H^3}{\bar{C}-\bar{C}_b} S \sqrt{A K}\right),
\end{aligned}
$$

(a)はOptimism，(b)は単純な変形，(c)は$\epsilon_k$についてのバウンドを使ってます．

この１項目をバウンドしましょう．
価値差分方程式より，

$$
\begin{aligned}
& \sum_{k=K_0}^K\left(V_{\bar{r}_k}^{\pi_k}(P)-V_{\bar{r}_k}^{\pi_k}\left(P_k\right)\right) \\
& {=} \sum_{k=K_0}^K \sum_{h=1}^H \mathbb{E}\left[\sum_{s^{\prime}}\left(P_h-P_{h, k}\right)\left(s^{\prime} \mid s_{h, k}, a_{h, k}\right) V_{\bar{r}_k, h+1}^{\pi_k}\left(s^{\prime} ; P_k\right) \mid \pi_k, P, \mathcal{F}_{k-1}\right] \\
& \leq \sum_{k=K_0} \sum_{h=1}^H \mathbb{E}\left[\sum_{s^{\prime}} \mid\left(P_h-P_{h, k}\right)\left(s^{\prime} \mid s_{h, k}, a_{h, k}\right)\left\|V_{r, h+1}^{\pi_k}\left(s^{\prime} ; P\right)\right\| \pi_k, P, \mathcal{F}_{k-1}\right]+ \\
& \sum_{k=K_0}^K \sum_{h=1}^H \mathbb{E}\left[\mid \sum_{s^{\prime}}\left(P_h-P_{h, k}\right)\left(s^{\prime} \mid s_{h, k}, a_{h, k}\right)\left(V_{\bar{r}_k, h+1}^{\pi_k}\left(s^{\prime} ; P_k\right)-V_{r, h+1}^{\pi_k}\left(s^{\prime} ; P\right)\right) \| \pi_k, P, \mathcal{F}_{k-1}\right]
\end{aligned}
$$

が成り立ちます．一項目をバウンドします．

$$
\begin{aligned}
& \sum_{k=K_0}^K \sum_{h=1}^H \mathbb{E}\left[\sum_{s^{\prime}}\left|\left(P_h-P_{h, k}\right)\left(s^{\prime} \mid s_{h, k}, a_{h, k}\right)\right| \mid V_{r, h+1}^{\pi_k}\left(s^{\prime} ; P\right) \| \pi_k, P, \mathcal{F}_{k-1}\right] \\
& \leq \sum_{k=K_0}^K \epsilon_k^{\pi_k}(P) {\leq} \tilde{O}\left(S \sqrt{A H^4 K^{\prime}}\right),
\end{aligned}
$$

２項目をバウンドしましょう．
Niceエピソードの話を使えば，

$$
\begin{aligned}
\sum_{k=K_0}^K \sum_{h=1}^H \mathbb{E} & {\left[\mid \sum_{s^{\prime}}\left(P_h-P_{h, k}\right)\left(s^{\prime} \mid s_{h, k}, a_{h, k}\right)\left(V_{\bar{r}_k, h+1}^{\pi_k}\left(s^{\prime} ; P_k\right)-V_{r, h+1}^{\pi_k}\left(s^{\prime} ; P\right)\right) \| \pi_k, P, \mathcal{F}_{k-1}\right] } \\
& =\sum_{k, h, s, a} w_h^{\pi_k}(s, a ; P) \sum_{s^{\prime}}\left(P_h-P_{h, k}\right)\left(s^{\prime} \mid s_{h, k}, a_{h, k}\right)\left(V_{\bar{r}_k, h+1}^{\pi_k}\left(s^{\prime} ; P_k\right)-V_{r, h+1}^{\pi_k}\left(s^{\prime} ; P\right)\right)\\
& \leq \underbrace{\sum_{k, h, s, a} w_h^{\pi_k}(s, a ; P) \sum_{s^{\prime}} \frac{\sqrt{P_h\left(s^{\prime} \mid s, a\right)}}{\sqrt{n_{h, k}(s, a) \vee 1}}\left|V_{\bar{r}_k, h+1}^{\pi_k}\left(s^{\prime} ; P_k\right)-V_{r, h+1}^{\pi_k}\left(s^{\prime} ; P\right)\right|}_{(A)} \\
&\quad +\underbrace{\sum_{k, h, s, a} w_h^{\pi_k}(s, a ; P) \frac{1}{n_{h, k}(s, a) \vee 1}\left|V_{\bar{r}_k, h+1}^{\pi_k}\left(s^{\prime} ; P_k\right)-V_{r, h+1}^{\pi_k}\left(s^{\prime} ; P\right)\right|}_{(B)},
\end{aligned}
$$

が成り立ちます．（A）と（B）はNiceエピソードの話を使えばうまいことバウンドできます．

これ以降は変形しまくればいけます．